In [4]:
import sys
import tkinter
import tkinter.messagebox
from tkintermapview import TkinterMapView
import pandas as pd
from pyswip import Prolog

ImportError: Could not find the SWI-Prolog library in this platform. If you are sure it is installed, please open an issue.

In [24]:
import import_ipynb
import extract

In [29]:
dest_df = pd.read_csv('Destinations.csv')
dest_df.head(10)

,Destinations,country,region,Climate,Budget,Activity,Demographics,Duration,Cuisine,History,Natural Wonder,Accommodation,Language
0,Tokyo,Japan,East Asia,Temperate,High,Cultural,Solo,Long,Asian,Modern,Mountains,Luxury,Japanese
1,Ottawa,Canada,North America,Cold,Medium,Adventure,Family-friendly,Medium,European,Modern,Forests,Mid-range,English
2,Mexico City,Mexico,North America,Temperate,Low,Cultural,Senior,Short,Latin American,Ancient,Mountains,Budget,Spanish
3,Rome,Italy,Southern Europe,Temperate,High,Cultural,Solo,Medium,European,Ancient,Beaches,Luxury,Italian
4,Brasilia,Brazil,South America,Tropical,Low,Adventure,Family-friendly,Long,Latin American,Modern,Beaches,Budget,Portuguese
5,Canberra,Australia,Oceania,Temperate,High,Adventure,Family-friendly,Long,Western,Modern,Beaches,Luxury,English
6,New Delhi,India,South Asia,Tropical,Low,Cultural,Solo,Long,Asian,Ancient,Mountains,Budget,Hindi
7,Pretoria,South Africa,Africa,Temperate,Medium,Adventure,Solo,Medium,African,Modern,Forests,Mid-range,English
8,Madrid,Spain,Southern Europe,Temperate,Medium,Cultural,Senior,Short,European,Ancient,Beaches,Mid-range,Spanish
9,Moscow,Russia,Eastern Europe,Cold,Low,Cultural,Senior,Long,Eastern European,Medieval,Mountains,Budget,Russian


In [21]:
features = list(map(sanatize, dest_df.columns))
dest_features = {}
values = dest_df.values
for val in values:
    dest_features[val[0]] = {}
    for i in range(1, len(features)):
        dest_features[val[0]][features[i]] = val[i]

In [22]:
dest_features

{'Tokyo': {'country': 'Japan',
  'region': 'East Asia',
  'climate': 'Temperate',
  'budget': 'High',
  'activity': 'Cultural',
  'demographics': 'Solo',
  'duration': 'Long',
  'cuisine': 'Asian',
  'history': 'Modern',
  'natural wonder': 'Mountains',
  'accommodation': 'Luxury',
  'language': 'Japanese'},
 'Ottawa': {'country': 'Canada',
  'region': 'North America',
  'climate': 'Cold',
  'budget': 'Medium',
  'activity': 'Adventure',
  'demographics': 'Family-friendly',
  'duration': 'Medium',
  'cuisine': 'European',
  'history': 'Modern',
  'natural wonder': 'Forests',
  'accommodation': 'Mid-range',
  'language': 'English'},
 'Mexico City': {'country': 'Mexico',
  'region': 'North America',
  'climate': 'Temperate',
  'budget': 'Low',
  'activity': 'Cultural',
  'demographics': 'Senior',
  'duration': 'Short',
  'cuisine': 'Latin American',
  'history': 'Ancient',
  'natural wonder': 'Mountains',
  'accommodation': 'Budget',
  'language': 'Spanish'},
 'Rome': {'country': 'Italy'

In [4]:
map_df = pd.read_csv('Adjacency_matrix.csv', index_col="Destinations")
map_df.head()

,Tokyo,Ottawa,Mexico City,Rome,Brasilia,Canberra,New Delhi,Pretoria,Madrid,Moscow,...,Bursa,Munich,Hamburg,Frankfurt,Alexandria,Luxor,Aswan,Christchurch,Queenstown,Rotorua
Destinations,,,,,,,,,,,,,,,,,,,,,
Tokyo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
Ottawa,1,0,1,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
Mexico City,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
Rome,1,0,0,0,1,1,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
Brasilia,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [5]:
class App(tkinter.Tk):

    APP_NAME = "map_view_demo.py"
    WIDTH = 800
    HEIGHT = 750  # This is now the initial size, not fixed.

    def __init__(self, *args, **kwargs):
        tkinter.Tk.__init__(self, *args, **kwargs)

        self.title(self.APP_NAME)
        self.geometry(f"{self.WIDTH}x{self.HEIGHT}")

        # Configure the grid
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)  # Text area and submit button combined row
        self.grid_rowconfigure(1, weight=4)  # Map row

        # Upper part: Text Area and Submit Button
        self.text_area = tkinter.Text(self, height=5)  # Reduced height for text area
        self.text_area.grid(row=0, column=0, pady=(10, 0), padx=10, sticky="nsew")

        self.submit_button = tkinter.Button(self, text="Submit", command=self.process_text)
        self.submit_button.grid(row=0, column=0, pady=(0, 10), padx=10, sticky="se")  # Placed within the same cell as text area

        # Lower part: Map Widget
        self.map_widget = TkinterMapView(self)
        self.map_widget.grid(row=1, column=0, sticky="nsew")

        self.marker_list = []  # Keeping track of markers
        self.marker_path = None


    def __init__(self, *args, **kwargs):
        tkinter.Tk.__init__(self, *args, **kwargs)

        self.title(self.APP_NAME)
        self.geometry(f"{self.WIDTH}x{self.HEIGHT}")

        # Configure the grid
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)  # Text area can expand/contract.
        self.grid_rowconfigure(1, weight=0)  # Submit button row; doesn't need to expand.
        self.grid_rowconfigure(2, weight=3)  # Map gets the most space.

        # Upper part: Text Area and Submit Button
        self.text_area = tkinter.Text(self)
        self.text_area.grid(row=0, column=0, pady=10, padx=10, sticky="nsew")
        
        self.submit_button = tkinter.Button(self, text="Submit", command=self.process_text)
        self.submit_button.grid(row=1, column=0, pady=10, sticky="ew")

        # Lower part: Map Widget
        self.map_widget = TkinterMapView(self)
        self.map_widget.grid(row=2, column=0, sticky="nsew")

        self.marker_list = []  # Keeping track of markers

    def check_connections(self, results):
        print('result2 ', results)
        locations = []
        for result in results:
            city  = result["City"]
            locations.append(city)
            # TODO 5: create the knowledgebase of the city and its connected destinations using Adjacency_matrix.csv

        return locations

    def process_text(self):
        """Extract locations from the text area and mark them on the map."""
        text = self.text_area.get("1.0", "end-1c")  # Get text from text area
        features = self.extract_features(text)  # Extract locations (you may use a more complex method here)

        print(f"{features=}")


        # TODO 4: create the query based on the extracted features of user desciption 
        ################################################################################################
        properties = ""
        for i, col in enumerate(dest_df.columns):
            if i == 0:
                continue
            properties += f'{list(features[sanatize(col)])[0] if sanatize(col) in features else "_"}'

            if i != len(dest_df.columns) - 1:
                properties += ", "
        query = f"destination(City, {properties})"

        print(f"{query=}")
        
        results = list(prolog.query(query))
        print(results)
        locations = self.check_connections(results)
        # TODO 6: if the number of destinations is less than 6 mark and connect them 
        ################################################################################################
        print(locations)
        # locations = ['mexico_city','rome' ,'brasilia']
        self.mark_locations(locations)

    def mark_locations(self, locations):
        """Mark extracted locations on the map."""
        for address in locations:
            marker = self.map_widget.set_address(address, marker=True)
            if marker:
                self.marker_list.append(marker)
        self.connect_marker()
        self.map_widget.set_zoom(1)  # Adjust as necessary, 1 is usually the most zoomed out


    def connect_marker(self):
        position_list = []

        for marker in self.marker_list:
            position_list.append(marker.position)

        if hasattr(self, 'marker_path') and self.marker_path is not None:
            self.map_widget.delete(self.marker_path)

        if len(position_list) > 0:
            self.marker_path = self.map_widget.set_path(position_list)

    def extract_features(self, text):
        """Extract locations from text. A placeholder for more complex logic."""

        return extract.extract_features(text)

    def point_location(location, features):
        point = 0
        for feature in features:
            if dest_features[location][feature] in features[feature]:
                point += 1
        return point
                

    def start(self):
        self.mainloop()


In [14]:
def sanatize(string):
    return string.lower().replace('\'', '').replace('.', '').split(',')[0]

In [15]:
def fact_exists(prolog, fact):
    # Check if the fact exists in the knowledge base
    return list(prolog.query(fact))

In [16]:
def assert_if_not_exists(prolog, fact):
    # Check if the fact already exists
    if not fact_exists(prolog, fact):
        # If it doesn't exist, assert the fact
        prolog.assertz(fact)

In [9]:
prolog = Prolog()

In [10]:
prolog.retractall("destination(_, _, _, _, _, _, _, _, _, _, _, _, _)")

for data in dest_df.values:
    assert_str = ''
    for i in range(len(data)):
        
        assert_str += '\'' + sanatize(data[i]) + '\''
        
        if i != len(data) - 1:
            assert_str += ', '    
            
    prolog.assertz(f"destination({assert_str})")

In [11]:
prolog.retractall("directly_connected(_, _)")
prolog.retractall("connected(_, _)")
prolog.retractall("connected2(_, _)")

all_cities = map_df.index

for ct1 in all_cities:
    for ct2 in all_cities:
        if ct2 == ct1:
            continue
        if map_df[ct1][ct2]:
            assert_if_not_exists(prolog, f"directly_connected('{sanatize(ct1)}', '{sanatize(ct2)}')")

prolog.assertz("connected(X, Y) :- directly_connected(X, Y)")
prolog.assertz("connected(X, Y) :- directly_connected(Y, X)")

prolog.assertz("connected2(X, Y) :- connected(X, Z), connected(Z, Y)")

In [12]:
result = list(prolog.query("connected2(C, 'ottawa')"))
visited = {}
count = 0
# print(result)
for res in result:
    if res['C'] in visited:
        continue

    visited[res['C']] = True
    count += 1
print(visited)

{'tokyo': True, 'ottawa': True, 'mexico city': True, 'rome': True, 'brasilia': True, 'canberra': True, 'new delhi': True, 'pretoria': True, 'madrid': True, 'moscow': True, 'berlin': True, 'bangkok': True, 'stockholm': True, 'cairo': True, 'wellington': True, 'ankara': True, 'buenos aires': True, 'kathmandu': True, 'athens': True, 'nairobi': True, 'lisbon': True, 'oslo': True, 'jakarta': True, 'hanoi': True, 'dublin': True, 'beijing': True, 'paris': True, 'santiago': True, 'rabat': True, 'singapore': True, 'lima': True, 'helsinki': True, 'prague': True, 'manila': True, 'brussels': True, 'kuala lumpur': True, 'amsterdam': True, 'washington dc': True, 'london': True, 'seoul': True, 'bogota': True, 'warsaw': True, 'jerusalem': True, 'kyiv': True, 'vienna': True, 'copenhagen': True, 'dhaka': True, 'montevideo': True, 'bern': True, 'islamabad': True, 'budapest': True, 'zagreb': True, 'colombo': True, 'tehran': True, 'bratislava': True, 'cancun': True, 'venice': True, 'florence': True, 'milan

In [13]:
# TODO 2: extract unique features from the Destinations.csv and save them in a dictionary
################################################################################################

In [14]:
city_info = {}

query = 'destination(City, _, _, _, low, _, _, _, _, _, _, _, _)'
results = list(prolog.query(query))

In [15]:
if __name__ == "__main__":
    app = App()
    app.start()

In [32]:
p = extract.extract_features("I want go to medium budget to africa region")

In [33]:
p['region']

{'africa'}

In [34]:
def point_location(location, features):
    point = 0
    for feature in features:
        if dest_features[location][feature] in features[feature]:
            point += 1
    return point

In [35]:
point_location('pretoria', p)

KeyError: 'pretoria'

In [ ]:
dest_features[pretoria']